## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [26]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\\Weather_Database\\WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Whitecourt,CA,54.1502,-115.6855,52.20,53,100,3.36,overcast clouds
1,1,Comodoro Rivadavia,AR,-45.8667,-67.5000,49.86,40,0,20.71,clear sky
2,2,Ushuaia,AR,-54.8000,-68.3000,35.24,94,88,2.37,light rain
3,3,Okhotsk,RU,59.3833,143.3000,45.64,89,100,9.28,light rain
4,4,Taoudenni,ML,22.6783,-3.9836,91.04,9,5,10.16,clear sky


In [27]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 100


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Taoudenni,ML,22.6783,-3.9836,91.04,9,5,10.16,clear sky
18,18,San Patricio,US,28.0170,-97.5169,82.49,78,53,9.04,broken clouds
20,20,Diu,IN,20.7141,70.9822,82.27,78,87,3.36,overcast clouds
29,29,Butaritari,KI,3.0707,172.7902,83.88,74,5,15.88,clear sky
30,30,Georgetown,MY,5.4112,100.3354,87.73,70,40,11.50,scattered clouds
40,40,Airai,TL,-8.9266,125.4092,81.93,31,33,2.93,scattered clouds
41,41,Port Blair,IN,11.6667,92.7500,81.09,82,98,6.08,overcast clouds
43,43,Kavieng,PG,-2.5744,150.7967,85.37,68,67,8.61,broken clouds
48,48,Hithadhoo,MV,-0.6000,73.0833,82.83,74,99,14.97,overcast clouds
62,62,Alice Springs,AU,-23.7000,133.8833,90.95,12,30,9.22,scattered clouds


In [29]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        103
City           103
Country        103
Lat            103
Lng            103
Max Temp       103
Humidity       103
Cloudiness     103
Wind Speed     103
Description    103
dtype: int64

In [30]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=preferred_cities_df.dropna()

In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Taoudenni,ML,91.04,clear sky,22.6783,-3.9836,
18,San Patricio,US,82.49,broken clouds,28.0170,-97.5169,
20,Diu,IN,82.27,overcast clouds,20.7141,70.9822,
29,Butaritari,KI,83.88,clear sky,3.0707,172.7902,
30,Georgetown,MY,87.73,scattered clouds,5.4112,100.3354,
40,Airai,TL,81.93,scattered clouds,-8.9266,125.4092,
41,Port Blair,IN,81.09,overcast clouds,11.6667,92.7500,
43,Kavieng,PG,85.37,broken clouds,-2.5744,150.7967,
48,Hithadhoo,MV,82.83,overcast clouds,-0.6000,73.0833,
62,Alice Springs,AU,90.95,scattered clouds,-23.7000,133.8833,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.loc[(hotel_df["Hotel Name"] !="")]
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
18,San Patricio,US,82.49,broken clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
20,Diu,IN,82.27,overcast clouds,20.7141,70.9822,Hotel Relax Inn
29,Butaritari,KI,83.88,clear sky,3.0707,172.7902,Isles Sunset Lodge
30,Georgetown,MY,87.73,scattered clouds,5.4112,100.3354,Cititel Penang
40,Airai,TL,81.93,scattered clouds,-8.9266,125.4092,Juvinal Martin
...,...,...,...,...,...,...,...
657,Kattivakkam,IN,89.69,overcast clouds,13.2167,80.3167,Euronova Freezer
666,Turbat,PK,94.64,scattered clouds,26.0023,63.0440,Mubashar Quarters
677,Kulhudhuffushi,MV,80.73,overcast clouds,6.6221,73.0700,Hondaafushi Island Resort
680,Cayenne,GF,81.09,scattered clouds,4.9333,-52.3333,Hôtel Le Dronmi


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))